In [1]:
from kafka import KafkaConsumer
import json
import pandas as pd
from sqlalchemy import create_engine, text
import time
import os
from datetime import datetime
from urllib.parse import quote_plus
from sqlalchemy.types import NVARCHAR

In [2]:
ip_host = '10.222.222.95'
date_col = []

In [3]:
#Destination config
username_source = ''
password_source = ''
hostname_source = ''
port_source = ''
database_source = ''
prefix = ''
list_table_name = []
file_dir_table = './cdc_table.txt'
file_dir_dt_col = './dt_col.txt'
file_dir_config_source = './config_source.txt'

In [4]:
username_destination = ''
password_destination = ''
hostname_destination = ''
port_destination = ''
database_destination = ''
database_source = ''
file_dir_config_destination = './config_destination.txt'

In [5]:
def take_config_from_file(file_dir):
    with open(file_dir) as f:
        lines = f.readlines()
        f.close()
    return lines[1]

In [6]:
def take_table_name_from_file(file_dir):
    list_table_name = []
    with open(file_dir) as f:
        lines = f.readlines()
        f.close()
    list_table_tmp = lines[0].split(",")
    for item in list_table_tmp:
        if(item[0] == ' '):
            item = item[1:]
        list_table_name.append(item)
    f.close()
    return list_table_name

In [7]:
def take_dt_col_name_from_file(file_dir):
    date_col = []
    with open(file_dir) as f:
        lines = f.readlines()
        f.close()
    list_col_tmp = lines[0].split(",")
    for item in list_col_tmp:
        if(item[0] == ' '):
            item = item[1:]
        date_col.append(item)
    f.close()
    return date_col

In [8]:
def take_table_name(df):
    source = pd.DataFrame([df.loc[0,'source']])
    table_name = source.loc[0,'table']
    return table_name

In [9]:
#take topic name
def take_list_topic_name(list_table_name):
    list_topic_name = []
    for table_name in list_table_name:
        topic_name = f'{prefix}.{database_source}.{table_name}'
        list_topic_name.append(topic_name)
    return list_topic_name

In [10]:
#read json data to dataframe
def read_json_to_df(json):
    data = pd.DataFrame([json])
    return data

In [11]:
def take_time():
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    return dt_string

In [12]:
def insert_data(df,table_name):
    #Take new data
    cdc_table = pd.DataFrame([df.loc[0,'after']]) 
    txt_cols = cdc_table.select_dtypes(include = ['object']).columns.values.tolist()
    #Change nano epoch time to datetime
    for col in cdc_table.columns:
        if(col in date_col):
            try:
                epoch_time = cdc_table.loc[0, col]
                dt_obj = datetime.utcfromtimestamp(epoch_time/1000)
                cdc_table.loc[0, col] = dt_obj
            except Exception as e:
                cdc_table.loc[0,col] = None
                cdc_table[col] = pd.to_datetime(cdc_table[col],errors='coerce')
    
    cdc_table.to_sql(f'{table_name}', engine, if_exists='append', index = False, dtype = {col_name: NVARCHAR for col_name in txt_cols})
    
    f = open(f".\CDC_logs\cdc_log_{database_source}.txt", "a")
    f.write(f"{table_name}\tID\t{cdc_table.loc[0,'ID']}\t{take_time()}\tInsert\n")
    f.close()

In [13]:
def update_data(df,table_name):
    cdc_table = pd.DataFrame([df.loc[0,'after']])
    txt_cols = cdc_table.select_dtypes(include = ['object']).columns.values.tolist()
    #string set by loop
    set_str = "SET"
    for col in cdc_table.columns:
        set_str = set_str + f" {col} = t.{col},\n" #SQL command
        if(col in date_col):
            try:
                epoch_time = cdc_table.loc[0, col]
                dt_obj = datetime.utcfromtimestamp(epoch_time/1000)
                cdc_table.loc[0, col] = dt_obj
            except Exception as e:
                cdc_table.loc[0,col] = None
                cdc_table[col] = pd.to_datetime(cdc_table[col],errors='coerce')
            
    sql = f"UPDATE dbo.{table_name}\n" + set_str[:-2] + "\nFROM temp t " +f"WHERE dbo.{table_name}.ID = t.ID"
    
    #delete temp table after complete
    sql_del_temp = "Drop table temp"
        
    #create temp table to reduce time to reconize columns type
    cdc_table.to_sql('temp', engine, if_exists='replace', dtype = {col_name: NVARCHAR for col_name in txt_cols})

    #run sql command
    with engine.begin() as conn:
        conn.execute(text(sql))
        conn.execute(text(sql_del_temp))
        f = open(f".\CDC_logs\cdc_log_{database_source}.txt", "a")
        f.write(f"{table_name}\tID\t{cdc_table.loc[0,'ID']}\t{take_time()}\tUpdate\n")
        f.close()

In [14]:
def delete_data(df,table_name):
    cdc_table = pd.DataFrame([df.loc[0,'before']])
    
    sql = f"DELETE FROM dbo.{table_name} WHERE dbo.{table_name}.ID = {cdc_table.loc[0,'ID']}"

    with engine.begin() as conn:
        conn.execute(text(sql))
        f = open(f".\CDC_logs\cdc_log_{database_source}.txt", "a", encoding="utf-8")
        f.write(f"{table_name}\tID\t{cdc_table.loc[0,'ID']}\t{take_time()}\tDelete\n")
        f.close()

In [15]:
#Use file config to take config
username_source, password_source, hostname_source, port_source, database_source, prefix \
= take_config_from_file(file_dir_config_source).split('\t')

In [16]:
username_destination, password_destination, hostname_destination, port_destination, database_destination \
= take_config_from_file(file_dir_config_destination).split('\t')

In [17]:
#Use file table to take all table
list_table_name = take_table_name_from_file(file_dir_table)
#Use file columns to take all datetime columns
date_col = take_dt_col_name_from_file(file_dir_dt_col)

In [18]:
file_dir_log = f'CDC_logs/cdc_log_{database_source}.txt'
if (os.path.isfile(file_dir_log)) == False:
    f = open(f"{file_dir_log}", "a", encoding="utf-8")
    f.write("Table\tID\tcreate_at\ttype\n")
    f.close()

In [19]:
# Define server with port
bootstrap_servers = [f'{ip_host}:9092']

#define the connect engine
password_destination = quote_plus(password_destination)
engine = create_engine(f'mssql+pymssql://{username_destination}:{password_destination}@{hostname_destination}:{port_destination}/{database_destination}?charset=utf8') 

#define the topic list
list_topic = take_list_topic_name(list_table_name)

# Initialize consumer variable
consumer = KafkaConsumer (*list_topic, 
                          bootstrap_servers = bootstrap_servers,
                          auto_offset_reset='earliest', 
                          enable_auto_commit=True,
                          auto_commit_interval_ms=10000, 
                          group_id='cmis_to_dwh')


In [ ]:
# Read and print message from consumer
for msg in consumer:
    try:
        #Read data from comsumer
        data = json.loads(msg.value)
        df = read_json_to_df(data)
        #Take change data
        df = pd.DataFrame([df.loc[0,'payload']])
        #take table name
        table_name = take_table_name(df)
        #Check action
        if(df.loc[0,'before'] == None and df.loc[0,'after'] == None):
            print('error')
            continue
        elif (df.loc[0,'before'] == None):
            insert_data(df,table_name)
        elif (df.loc[0,'after'] == None):
            delete_data(df,table_name)
        else:
            update_data(df,table_name)
    #Temporary use try except to avoid bug when read null message
    except TypeError:
        time.sleep(0.001)